<a href="https://colab.research.google.com/github/KNUckle-llm/experiments/blob/main/RAG_advanced.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 개발 환경 확인
- GPU 환경을 확인하는 명령어 nivida-smi, L4 혹은 A100 인지 확인
- 고용량 RAM 용량 확인

In [ ]:
!free -h

               total        used        free      shared  buff/cache   available
Mem:            52Gi       1.6Gi        45Gi       4.0Mi       5.6Gi        50Gi
Swap:             0B          0B          0B


In [ ]:
!nvidia-smi

Fri Mar 28 06:36:13 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   34C    P8             11W /   72W |       0MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# 필요한 모듈이나 라이브러리 설치

- 해당 셀에서 필요한 모듈 혹은 라이브러리를 설치(Ctrl + M + A)

In [ ]:
!pip install -q chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 57.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 80.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 130.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.1 MB/s eta 0:00:

# VectorStore 로드
- 프로젝트에 사용되고 있는 VectorStore 로드

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 저장된 결과 보기
# 1. 필요한 모듈 임포트
from chromadb import PersistentClient

# 2. ChromaDB 클라이언트 연결
persist_dir = "/content/drive/MyDrive/chroma_index"  # 실제 저장된 경로로 설정
client = PersistentClient(path=persist_dir)

# 저장된 컬렉션 불러오기 (컬렉션 이름은 실제 생성한 이름으로!)
collection = client.get_or_create_collection(name="software_department")

# 전체 문서 + 메타데이터 + 임베딩 가져오기
results = collection.get(include=["documents", "metadatas", "embeddings"])

# 결과 출력
print(f"✅ 저장된 총 청크 수: {len(results['documents'])}")
for i in range(len(results['documents'])):
    print(f"🆔 ID: {results['ids'][i]}")
    print(f"📄 청크 내용: {results['documents'][i][:200]}...")  # 너무 길면 앞부분만
    print(f"📎 메타데이터: {results['metadatas'][i]}")
    print(f"🔢 임베딩 벡터 (일부): {results['embeddings'][i][:5]}...")  # 임베딩 앞부분 5개만 보기
    print("-" * 80)

✅ 저장된 총 청크 수: 108
🆔 ID: 소프트웨어전공 졸업규정_chunk_0
📄 청크 내용: 소프트웨어전공 졸업논문 규정 제1조(목적) 이 내규는 공주대학교 학칙(이하 “학칙”이라 함) 제94조 및 공주대학교 학사운영 규정(이하 “학사규정 ”이라 함) 제12장에 의거 소프트웨어전공 , 컴퓨터소프트웨어공학전공 및 멀티미디어공학전공의 졸업논문에 관한 세부사항을 규정함을 목적으로 한다. 제2조(졸업논문 제출자격 ) ① 최종학기 등록을 필한 자라야 한다....
📎 메타데이터: {'date': '2025-03-27', 'department': '공주대학교 소프트웨어학과', 'source_url': 'https://sw.kongju.ac.kr/synap/skin/doc.html?fn=ZD1180_1426_208960_3&rs=/synap/result/bbs/1426', 'type': 'pdf'}
🔢 임베딩 벡터 (일부): [ 0.06352967  0.0725045   0.05380703  0.05122475 -0.06273629]...
--------------------------------------------------------------------------------
🆔 ID: 소프트웨어전공 졸업규정_chunk_1
📄 청크 내용: 지 발표하고 , 종강전까지 인쇄본을 제출하여야한다 ② 단독연구만 을 인정한다 . ③ 지도교수는 졸업논문의 자료준비 , 작성 및 발표 등에 관한 사항을 지도한다 . ④ 졸업논문의 제출 부수는 3부(심사위원 2부, 학과사무실 1부) 제출한다 제6조(졸업논문 심사위원 ) ① 졸업논문 심사위원은 지도교수를 포함한 2인 이상으로 하며 지도 교수의 추천에 의하여 소속...
📎 메타데이터: {'date': '2025-03-27', 'department': '공주대학교 소프트웨어학과', 'source_url': 'https://sw.kongju.ac.kr/synap/skin/doc.html?fn=ZD1180_1426_208960_3&rs=/sy

# Advanced RAG
- Naive RAG에서 Retrieval 처리를 더 강화한 RAG 패러다임
![Advanced RAG](https://miro.medium.com/v2/resize:fit:1400/0*d8aZGfRkfcFVf4jA.png)

## **Retrieval**

- Lexical Search + Semantic Search -> Hybrid Search
- BM25 + Reciprocal Rank Fusion

### [**Okapi BM25**](https://en.wikipedia.org/wiki/Okapi_BM25)
- _BM_ 은 best matching의 약어
- 확률적 정보 검색(IR: Information Retrieval)에서 널리 쓰이는 문서 랭킹 함수
1. 쿼리에서 등장하는 단어가 문서에서 얼마나 자주 등장하는지 **(TF: Term Frequency)**
2. 그 단어가 전체 문서에서 얼마나 희귀한지 **(IDF: Inverse Document Frequency)**
3. 문서의 길이에 따라 가중치 부여 (길이가 긴 문서는 점수를 낮추는 방식)

</br>
$$
\text{score}(D, Q) = \sum_{i=1}^{n} \text{IDF}(q_i) \cdot
\frac{f(q_i, D) \cdot (k_1 + 1)}
{f(q_i, D) + k_1 \cdot \left(1 - b + b \cdot \frac{|D|}{\text{avgdl}}\right)}
$$
</br>

- $f(q_i, D)$ : 문서 $D$ 내의 특정 단어 $q_i$가 나타내는 횟수
- $|D|$ : 문서 $D$의 길이
- avgdl : 전체 문서의 평균 길이
- $k_1$과 $b$ : hyper parameters
  - $k_1$ : 키워드의 빈도에 따른 점수 영향력 억제, 무분별한 키워드 빈도 수 증가에 따른 중요도 향상 억제한다. (default 1.2 ~ 2.0)
  - $b$ : 문서 길이에 대한 중요도 제한, 짧은 문서에 특정 단어가 존재하면 중요도를 높게 평가한다. (default 0.75)

</br>
$$
\text{IDF}(q_i) = \ln \left( \frac{N - n(q_i) + 0.5}{n(q_i) + 0.5} + 1 \right)
$$
</br>

- $N$ : 전체 문서 수
- $n(q_i)$ : 단어 $q_i$를 포함하는 문서의 수
- IDF(Inverse Document Frequency)는 특정 단어 $q_i$가 모든 문서에 존재한다면 중요하지 않다고 여기고 특정 문서에만 존재하는 단어라면 중요하다고 평가한다. **IDF가 낮으면 중요도가 낮고, IDF가 높으면 중요도가 높다.**



In [11]:
# BM25 from Scratch
import math
from collections import Counter

class BM25:
  def __init__(self, corpus, k1=1.2, b=0.75):
    self.corpus = corpus
    self.k1 = k1
    self.b = b
    self.N = len(corpus)
    self.avgdl = sum(len(doc) for doc in corpus) / self.N
    self.df = self._calculate_df()
    self.idf = self._calculate_idf()

  # Python에서 함수 앞에 _는 private이라는 관용 표현, 외부에서 호출하지 않는다는 의미
  ## Document Frequency
  def _calculate_df(self):
    df = {}
    for doc in self.corpus:
      for word in set(doc):
        # 딕셔너리 df에 word 키가 이미 있으면 +1, 없으면 0 + 1로 새로 추가
        df[word] = df.get(word, 0) + 1
    return df

  #Inverted Document Frequency
  def _calculate_idf(self):
    idf = {}
    for term, df in self.df.items():
        idf[term] = math.log((self.N - df + 0.5) / (df + 0.5) + 1)
    return idf

  def get_scores(self, query, doc_index):
    doc = self.corpus[doc_index]
    doc_len = len(doc)
    tf = Counter(doc)
    score = 0.0

    for q in query:
        if q not in tf:
            continue
        idf = self.idf.get(q, 0)
        freq = tf[q]
        denom = freq + self.k1 * (1 - self.b + self.b * (doc_len / self.avgdl))
        score += idf * ((freq * (self.k1 + 1)) / denom)
    return score

  def get_top_k(self, query, k=3):
    scores = [(i, self.get_scores(query, i)) for i in range(self.N)]
    return sorted(scores, key=lambda x: x[1], reverse=True)[:k]

In [12]:
corpus = [
    "the cat in the hat".split(),
    "the quick brown fox".split(),
    "the lazy dog and the fox".split()
]

query = "fox and dog"

bm25 = BM25(corpus)
top_docs = bm25.get_top_k(query.split())

for idx, score in top_docs:
    print(f"Doc {idx}: {corpus[idx]} → Score: {score:.4f}")

Doc 2: ['the', 'lazy', 'dog', 'and', 'the', 'fox'] → Score: 2.2478
Doc 1: ['the', 'quick', 'brown', 'fox'] → Score: 0.5119
Doc 0: ['the', 'cat', 'in', 'the', 'hat'] → Score: 0.0000


### **상호 순위 조합(Reciprocal Ranking Fusion, RRF)**

# Evaluation